# Generate Neural Network Features for UWNDC2019 Model 

<br><br>
### Uses Alexnet and PyTorch

In [1]:
import numpy as np
import warnings
import pandas as pd
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from collections import namedtuple

warnings.simplefilter(action='ignore',category=FutureWarning)

datadir ='./Hackathon/v4-challenge/data/' # set this to the location of your data directory which
                   # should contain train.csv and stim.npy

In [2]:
df_now = pd.read_csv(datadir+'train.csv')
im_now = np.load(datadir+'stim.npy')

test_ims = im_now[50:,...]
train_ims = im_now[50:,...]

#calculate mean for image normalization
im_mean = np.mean(np.mean(im_now,axis=(1,2)),axis=0)
im_std = np.std(np.std(im_now,axis=(1,2)),axis=0)


In [3]:
class v4_dataset(Dataset):
    
    def __init__(self, df,ims, transform):
        
        self.resp_frame = df
        self.transform = transform
        self.ims = ims
        
    def __len__(self):
        return len(self.resp_frame)
    
    def __getitem__(self,idx):
        img = Image.fromarray(np.uint8(self.ims[idx,:,:,:]*255))
        responses = self.resp_frame.iloc[idx,1:].as_matrix()
        
        if self.transform is not None:
            img = self.transform(img)
            
        sample = {'image': img, 'responses': responses}
        
        return sample

tfm = transforms.Compose([transforms.Resize(224),transforms.ToTensor(), transforms.Normalize(mean=im_mean,std=im_std)])

test_set = v4_dataset(df_now,test_ims,tfm)
testloader = DataLoader(test_set,batch_size=4)

In [6]:
class AlexNet(torch.nn.Module):
    
    def __init__(self):
        super(AlexNet,self).__init__()
        features = list(models.alexnet(pretrained=True).features)
        self.features = nn.ModuleList(features).eval()
        
    def forward(self,x):
        results =[]
        for ii, cv_layer in enumerate(self.features):
            x = cv_layer(x)
            if ii in {0, 3, 6, 8, 10}: #just conv1-5
                results.append(x)
        
        an_outputs = namedtuple("AlexNetOutputs",['conv1','conv2','conv3','conv4','conv5'])
        return an_outputs(*results)
    
net = AlexNet()
net.eval()
conv_resps = dict()
device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
net.to(device)

for i, data in enumerate(testloader):
    
    inputs = data['image']
    inputs = inputs.to(device)
    outputs = net(inputs)
    
    if i == 0:
        for j, fld in enumerate(outputs._fields):
            #import pdb; pdb.set_trace()
            conv_resps[fld] = outputs[j].cpu().detach().numpy()
    else:
        for j, fld in enumerate(outputs._fields):
            conv_resps[fld] = np.vstack((conv_resps[fld],outputs[j].cpu().detach().numpy()))
            
    print('{}% done with test examples'.format(round((i+1)*100/len(testloader),2)))
    

0.72% done with test examples
1.45% done with test examples
2.17% done with test examples
2.9% done with test examples
3.62% done with test examples
4.35% done with test examples
5.07% done with test examples
5.8% done with test examples
6.52% done with test examples
7.25% done with test examples
7.97% done with test examples
8.7% done with test examples
9.42% done with test examples
10.14% done with test examples
10.87% done with test examples
11.59% done with test examples
12.32% done with test examples
13.04% done with test examples
13.77% done with test examples
14.49% done with test examples
15.22% done with test examples
15.94% done with test examples
16.67% done with test examples
17.39% done with test examples
18.12% done with test examples
18.84% done with test examples
19.57% done with test examples
20.29% done with test examples
21.01% done with test examples
21.74% done with test examples
22.46% done with test examples
23.19% done with test examples
23.91% done with test ex